In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import Workbook
from openpyxl import load_workbook
from openpyxl.styles import Alignment
from openpyxl.chart import BarChart, Reference, LineChart
from openpyxl.utils import get_column_letter
from openpyxl.drawing.image import Image
from __future__ import division
from pptx import Presentation
from pptx.chart.data import ChartData
from pptx.enum.chart import XL_CHART_TYPE
from pptx.util import Inches
from IPython.display import display

In [ ]:
# taken from http://opendata-ajuntament.barcelona.cat
df_file = pd.read_csv('25_noms_padro_any_genere_1996_2016.csv')
df_file.columns = ['ORDER', 'NAME', 'GENDER', 'YEAR', 'FREQUENCY']
df_men = df_file[df_file['GENDER'] == 'Home']
df_men.head(5)

In [ ]:
# for this case we only take statistics from men names. For this use, we will only pick the top 7 names
top_men_names = list(df_men.groupby('NAME')['FREQUENCY'].agg(['sum', np.size]).reset_index().sort_values(['size','sum'],ascending = [0,0]).head(7)['NAME'])
top_men_names

In [ ]:
# we build a dictionary with the names as a key, storing the dataframe, the row and the column where we want to insert the dataframe in a worksheet
dictNames = {elem : pd.DataFrame for elem in top_men_names}
i = 0
for key in dictNames.keys():
    dictNames[key] = {
        "dataframe" :df_file[df_file['NAME'] == key][['YEAR', 'ORDER', 'FREQUENCY']],
        "start_row" : 1+24*i,
        "start_col" : 1
    }
    i+=1
display(dictNames['ALEX']['dataframe'].head(3))

In [ ]:
# we create an empty workbook where we want to insert the dataframes and we drop them in their already specified positions.
# Unless we do it in this way, we can only insert one dataframe per worksheet
wb = Workbook()
doc_name = 'excel file.xlsx'
writer = pd.ExcelWriter(doc_name, engine='openpyxl')
writer.book = wb
writer.sheets = dict((ws.title, ws) for ws in wb.worksheets)

for key in dictNames.keys():
    df = dictNames[key]['dataframe']
    df.to_excel(writer,
                sheet_name='sheet',
                header=True,index=False,
                index_label=None,
                startrow=dictNames[key]['start_row'],
                startcol=dictNames[key]['start_col'])
wb.remove_sheet(wb.get_sheet_by_name('Sheet'))
writer.save()

In [ ]:
ws = wb.active
ws.title = 'better sheet'

for key in dictNames.keys():
    
    dataframe = dictNames[key]['dataframe']
    start_col = dictNames[key]['start_col']
    start_row = dictNames[key]['start_row']

    #we add a header with the name over the table, merge its cells and apply a prebuilt style
    header_start = get_column_letter(start_col+1)+str(start_row)
    header_end = get_column_letter(start_col+len(dataframe.columns))+str(start_row)
    ws[header_start].value = key
    
    ws.merge_cells(header_start+':'+header_end)
    
    ws[header_start].style = 'Accent2'
    ws[header_start].alignment = Alignment(horizontal='center')

# we adjust the width of the columns in order to match its content
for col in ws.columns:
    max_length = 0
    column = col[0].column # Get the column name
    for cell in col:
        # try-catch to avoid error on empty cells
        try: 
            if len(str(cell.value)) > max_length:
                max_length = len(cell.value)
        except:
            pass
    adjusted_width = (max_length + 2) * 1.2
    ws.column_dimensions[column].width = adjusted_width

wb.save('better excel file.xlsx')

In [ ]:
ws = wb.active
ws.title = 'even better sheet'

for key in dictNames.keys():
    
    dataframe = dictNames[key]['dataframe']
    start_col = dictNames[key]['start_col']
    start_row = dictNames[key]['start_row']
    
    
    chart1 = LineChart()
    data1 = Reference(ws,
                      min_col = start_col+2,
                      min_row = start_row+1,
                      max_col = start_col+2,
                      max_row = start_row+1+len(dataframe))
    cats1 = Reference(ws,
                      min_col = start_col+1,
                      min_row = start_row+2,
                      max_row = start_row+1+len(dataframe))
    chart1.add_data(data1, titles_from_data=True)
    chart1.set_categories(cats1)
    chart1.x_axis.title = ws[get_column_letter(start_col+1)+str(start_row+1)].value
    chart1.y_axis.title = ws[get_column_letter(start_col+2)+str(start_row+1)].value
    # chart1.y_axis.majorGridlines = None
    chart1.title = key
    ws.add_chart(chart1, get_column_letter(start_col+5)+str(start_row+1))
    
    chart2 = BarChart()
    data2 = Reference(ws, 
                       min_col = start_col+3,
                       min_row = start_row+1,
                       max_col = start_col+3,
                       max_row = start_row+1+len(dataframe)
                      )
    cats2 = Reference(ws,
                      min_col = start_col+1,
                      min_row = start_row+2,
                      max_row = start_row+1+len(dataframe))
    chart2.add_data(data2, titles_from_data=True)
    chart2.set_categories(cats2)
    chart2.x_axis.title = ws[get_column_letter(start_col+1)+str(start_row+1)].value
    chart2.y_axis.title = ws[get_column_letter(start_col+3)+str(start_row+1)].value
    chart2.title = key
    ws.add_chart(chart2, get_column_letter(start_col+15)+str(start_row+1))
    
    wb.save('even better excel file.xlsx')

In [ ]:
wb = load_workbook('even better excel file.xlsx')
ws = wb.active
ws.title = 'even more better sheet'



for key in dictNames.keys():
    
    dataframe = dictNames[key]['dataframe']
    start_col = dictNames[key]['start_col']
    start_row = dictNames[key]['start_row']

    chart1 = LineChart()
    data1 = Reference(ws,
                      min_col = start_col+2,
                      min_row = start_row+1,
                      max_col = start_col+2,
                      max_row = start_row+1+len(dataframe))
    cats1 = Reference(ws,
                      min_col = start_col+1,
                      min_row = start_row+2,
                      max_row = start_row+1+len(dataframe))
    chart1.add_data(data1, titles_from_data=True)
    chart1.set_categories(cats1)
    chart1.x_axis.title = ws[get_column_letter(start_col+1)+str(start_row+1)].value
    chart1.y_axis.title = ws[get_column_letter(start_col+2)+str(start_row+1)].value
    chart1.y_axis.majorGridlines = None
    chart1.title = key
    
    # for the second chart, a barchart in this case, we don't asign the categories, since it will use the same as chart1
    chart2 = BarChart()
    chart2.y_axis.title = ws[get_column_letter(start_col+3)+str(start_row+1)].value
    chart2.y_axis.axId = 200
    data2 = Reference(ws, 
                       min_col = start_col+3,
                       min_row = start_row+1,
                       max_col = start_col+3,
                       max_row = start_row+1+len(dataframe)
                      )
    chart2.add_data(data2, titles_from_data=True)
    
    # we move the y_axis in chart1 to the right by making it cross the y_axis at its maximum value (2016 in this example)
    chart1.y_axis.crosses = "max"
        
    # we can add the two charts together by adding them
    chart1+=chart2
    chart1.height = 10
    chart1.width = 20
    ws.add_chart(chart1, get_column_letter(start_col+5)+str(start_row+1))
    
wb.save('even much better excel file.xlsx')

In [ ]:
wb = load_workbook('even better excel file.xlsx')
ws = wb.active
ws.title = 'the best sheet'

for key in dictNames.keys():
    
    dataframe = dictNames[key]['dataframe']
    start_col = dictNames[key]['start_col']
    start_row = dictNames[key]['start_row']

    chart1 = LineChart()
    data1 = Reference(ws,
                      min_col = start_col+2,
                      min_row = start_row+1,
                      max_col = start_col+2,
                      max_row = start_row+1+len(dataframe))
    cats1 = Reference(ws,
                      min_col = start_col+1,
                      min_row = start_row+2,
                      max_row = start_row+1+len(dataframe))
    chart1.add_data(data1, titles_from_data=True)
    chart1.set_categories(cats1)
    chart1.x_axis.title = ws[get_column_letter(start_col+1)+str(start_row+1)].value
    chart1.y_axis.title = ws[get_column_letter(start_col+2)+str(start_row+1)].value
    chart1.y_axis.majorGridlines = None
    chart1.title = key
    
    # for the second chart, a barchart in this case, we don't asign the categories, since it will use the same as chart1
    chart2 = BarChart()
    chart2.y_axis.title = ws[get_column_letter(start_col+3)+str(start_row+1)].value
    chart2.y_axis.axId = 200
    data2 = Reference(ws, 
                       min_col = start_col+3,
                       min_row = start_row+1,
                       max_col = start_col+3,
                       max_row = start_row+1+len(dataframe)
                      )
    chart2.add_data(data2, titles_from_data=True)
    
    # we move the y_axis in chart1 to the right by making it cross the y_axis at its maximum value (2016 in this example)
    chart1.y_axis.crosses = "max"
        
    # we can add the two charts together by adding them
    chart1+=chart2
    chart1.height = 10
    chart1.width = 20
    ws.add_chart(chart1, get_column_letter(start_col+5)+str(start_row+1))

    #########################################
    ##code for inserting images
    
    
    # img = Image('images\\'+key+'.jpg')
    # img_position = get_column_letter(start_col+17)+str(start_row+2)
    # ws.add_image(img, img_position)


wb.save('the best excel file.xlsx')

In [ ]:
df_men_summary = df_file[df_file['GENDER'] == 'Home'][['NAME','FREQUENCY', 'YEAR']]

prs = Presentation('layout.pptx')


# we insert a cover for the presentation
title_slide_layout = prs.slide_layouts[0]
slide = prs.slides.add_slide(title_slide_layout)
title = slide.shapes.title
title.text = "Most frequent men names from 1996 to 2016"



# we create a summary slide for each year
for i in range (1996, 2016):
    df_slide = df_men_summary[df_men_summary['YEAR']== i ]
    slide = prs.slides.add_slide(prs.slide_layouts[1])
    # define chart data 
    chart_data = ChartData()
    chart_data.categories = list(df_slide['NAME'])
    chart_data.add_series('Series 1', list(df_slide['FREQUENCY']))
    
    title = slide.shapes.title
    title.text = 'Name frequency for year ' + str(i)    
    # add chart to slide 
    x, y, cx, cy = Inches(0), Inches(2), Inches(10), Inches(4.5)
    slide.shapes.add_chart(XL_CHART_TYPE.COLUMN_CLUSTERED, x, y, cx, cy, chart_data)

prs.save('Meetup presentation.pptx')